In [1]:
incompatibilidades = []
tiempos_lavado = {}
cantidad_prendas = 0
cantidad_incompatibilidades = 0

with open('datos.txt', 'r') as file:
    for line in file:
        if line.startswith('p'):
            _, _,cantidad_prendas, cantidad_incompatibilidades = line.split()
        if line.startswith('e'):
            _, prenda1, prenda2 = line.split()
            incompatibilidades.append((int(prenda1), int(prenda2)))
        if line.startswith('n'):
            _, numero_prenda, tiempo_lavado = line.split()
            tiempos_lavado[int(numero_prenda)] = int(tiempo_lavado)

cantidad_prendas = int(cantidad_prendas)
cantidad_incompatibilidades = int(cantidad_incompatibilidades)

print("Incompatibilidades: ", incompatibilidades)
print("Tiempos lavado: ", tiempos_lavado)

print("Chequeo que tengan el largo correcto: ")
print(f"Incompatibilidades: {cantidad_incompatibilidades == len(incompatibilidades)}")
print(f"Tiempos lavado: {cantidad_prendas == len(tiempos_lavado)}")



Incompatibilidades:  [(1, 10), (1, 19), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 17), (2, 11), (2, 20), (2, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 10), (2, 18), (2, 1), (3, 12), (3, 10), (3, 17), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 11), (3, 19), (3, 2), (3, 1), (4, 13), (4, 11), (4, 18), (4, 5), (4, 6), (4, 7), (4, 8), (4, 12), (4, 20), (4, 3), (4, 2), (4, 1), (5, 14), (5, 12), (5, 19), (5, 6), (5, 7), (5, 8), (5, 13), (5, 4), (5, 3), (5, 2), (5, 1), (6, 15), (6, 13), (6, 20), (6, 7), (6, 8), (6, 14), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (7, 16), (7, 14), (7, 8), (7, 15), (7, 6), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (8, 15), (8, 16), (8, 7), (8, 6), (8, 5), (8, 4), (8, 3), (8, 2), (8, 1), (9, 18), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 2), (9, 1), (10, 19), (10, 17), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 18), (10, 9), (10, 3), (10, 2), (10, 1), (11, 20), (11, 18), (

In [2]:
def son_incompatibles(prenda1, prenda2, incompatibilidades):
    return (prenda1, prenda2) in incompatibilidades or (prenda2, prenda1) in incompatibilidades

def asignar_prenda_en_lavados(tiempos, incompatibilidades):
    prendas_ordenadas = sorted(tiempos, key=tiempos.get, reverse=True)
    lavados = {}
    for prenda in prendas_ordenadas:
        lavado_asignado = None
        menor_tiempo = float('inf')
        for lavado, prendas_en_lavado in lavados.items():
            if all(not son_incompatibles(prenda, p, incompatibilidades) for p in prendas_en_lavado):
                tiempo_prependa = tiempos[prenda]
                if tiempo_prependa < menor_tiempo:
                    lavado_asignado = lavado
                    menor_tiempo = tiempo_prependa
        if lavado_asignado is None:
            lavados[len(lavados)+1] = [prenda]
        else:
            lavados[lavado_asignado].append(prenda)
    return lavados

resultado = asignar_prenda_en_lavados(tiempos_lavado, incompatibilidades)

suma_total = 0
for lavado, prendas in resultado.items():
    tiempo_lavado = max(tiempos_lavado[prenda] for prenda in prendas)
    suma_total += tiempo_lavado
    print(f"Lavado {lavado}: {prendas}, Tiempo Total: {tiempo_lavado}")
print(f"suma total tiempo: {suma_total}")


Lavado 1: [2, 17, 12], Tiempo Total: 10
Lavado 2: [7, 9, 20], Tiempo Total: 10
Lavado 3: [3, 16, 18], Tiempo Total: 9
Lavado 4: [6, 10], Tiempo Total: 9
Lavado 5: [1, 11], Tiempo Total: 8
Lavado 6: [4, 15, 19], Tiempo Total: 7
Lavado 7: [13, 8], Tiempo Total: 5
Lavado 8: [5], Tiempo Total: 3
Lavado 9: [14], Tiempo Total: 1
suma total tiempo: 62


In [3]:
# Formateo la solucion
with open('solucion.txt', 'w') as file:
    for lavado, prendas in resultado.items():
        for prenda in prendas:
            file.write(f"{prenda} {lavado}\n")